In [2]:
import numpy as np
import matplotlib.pyplot as plt

You will identify if a mushroom is poisonous or not using the following training data of 10 mushrooms:

| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |

First one-hot encode the features:

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

- `X_train` contains three features for each sample
- `y_train` is whether the mushroom is edible (1 = edible, 0 = poisonous)

In [3]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [4]:
# Takes a numpy array that indicates whether the example is edible or poisonous
# Compute p1 which is the fraction that are edible and calculate entropy H(p1)

def compute_entropy(y):
    if len(y) == 0:
        return 0
    p1 = np.sum(y)/len(y)
    if p1 == 0 or p1 == 1: # to handle 0log0
        return 0
    return -p1*np.log2(p1) - (1-p1)*np.log2(1-p1)

print("Entropy at root node: ", compute_entropy(y_train)) 

Entropy at root node:  1.0


Let's refer to each example using an index and each feature as well. For example, brown cap would be feature 0, and if the value is 0 it means it has a red cap.

For each time splitting the data, we will not use the actual data but instead indices referencing to the X_train dataset. This allows for recursion since we don't always work with the root_node.

| Index | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:-----:|:---------:|:--------------------:|:--------:|:------:|
|   0   |     1     |           1          |     1    |    1   |
|   1   |     1     |           0          |     1    |    1   |
|   2   |     1     |           0          |     0    |    0   |
|   3   |     1     |           0          |     0    |    0   |
|   4   |     1     |           1          |     1    |    1   |
|   5   |     0     |           1          |     1    |    0   |
|   6   |     0     |           0          |     0    |    0   |
|   7   |     1     |           0          |     1    |    1   |
|   8   |     0     |           1          |     0    |    1   |
|   9   |     1     |           0          |     0    |    0   |

#### Decision Tree Refresher

- Start with all examples at the root node
- Calculate information gain for splitting on all possible features, and pick the highest
- Split dataset according to the selected feature, and create left and right branches
- Keep repeating splitting process until stopping criteria is met

In [5]:
# Using indices is more efficient than using the actual data
# This function will work with ANY subset of the data just using indices

def split_dataset(X, node_indices, feature_index):
    left_indices, right_indices = [], []
    for i in node_indices:
        if X[i][feature_index] == 0:
            right_indices.append(i)
        else:
            left_indices.append(i)
    return left_indices, right_indices

root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
left_indices, right_indices = split_dataset(X_train, root_indices, feature_index=0)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


In [6]:
def compute_information_gain(X, y, node_indices, feature_index):
    left_indices, right_indices = split_dataset(X, node_indices, feature_index)
    H_node = compute_entropy(y[node_indices])
    H_left, H_right = compute_entropy(y[left_indices]), compute_entropy(y[right_indices])
    w_left = len(left_indices)/len(node_indices)
    w_right = len(right_indices)/len(node_indices)
    return H_node - (w_left*H_left + w_right*H_right)

info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature_index=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

Information Gain from splitting the root on brown cap:  0.034851554559677034


In [7]:
def get_best_split(X, y, node_indices):
    features = X.shape[1]
    best_feature = -1
    best_gain = 0
    for i in range(features):
        info_gain = compute_information_gain(X, y, node_indices, i)
        if info_gain > best_gain:
            best_gain = info_gain
            best_feature = i
    return best_feature

best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


In the code below, we will use a more advanced implementation to build and use the decision tree. **Please make sure you fully understand the code below, it is a good data structure and recursion practice!**

In [8]:
class Node:
    def __init__(self, feature, indices, class_prediction=None):
        self.feature = feature # if this is none then it is a leaf node
        self.indices = indices
        self.class_prediction = class_prediction
        self.left = None
        self.right = None

# the stopping criteria we use is when maximum depth is 2
def build_tree_recursive(X, y, node_indices, max_depth, depth):
    if depth == max_depth:
        leaf_value = int(np.sum(y_train[node_indices]) > len(node_indices)/2)
        return Node(None, node_indices, class_prediction=leaf_value)
    
    feature = get_best_split(X, y, node_indices)
    left_indices, right_indices = split_dataset(X, node_indices, feature)

    node = Node(feature, node_indices)
    node.left = build_tree_recursive(X, y, left_indices, max_depth, depth + 1)
    node.right = build_tree_recursive(X, y, right_indices, max_depth, depth + 1)

    return node # return the root node

root_node = build_tree_recursive(X_train, y_train, root_indices, max_depth=2, depth=0)

def print_tree(node):
    if node.feature is None:
        print("Leaf node")
    else: 
        print("Splitting on feature %d" % node.feature)
        print_tree(node.left)
        print_tree(node.right)

print_tree(root_node)

Splitting on feature 2
Splitting on feature 0
Leaf node
Leaf node
Splitting on feature 1
Leaf node
Leaf node


In [9]:
# now we use the built tree to make predictions

def predict(node, x):
    if node.feature is None:
        return node.class_prediction

    if x[node.feature] == 0: # very simple splitting rule!
        return predict(node.right, x)
    else:
        return predict(node.left, x)
    
X_test = np.array([[1,1,0],[1,0,1],[0,1,1],[0,0,0],[1,0,0]])
y_pred = np.zeros(len(X_test))
for i, x in enumerate(X_test):
    y_pred[i] = predict(root_node, x)

print("Predictions: ", y_pred)

Predictions:  [1. 1. 0. 0. 0.]
